In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import glob
from skimage.transform import resize
import os
import pandas as pd

In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import tensorflow as tf
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

## Loading the data

In [4]:
x_train = np.load("train_data.npy")
x_test = np.load("test_data.npy")

In [11]:
y_train = pd.read_csv("train_label.csv")
y_test = pd.read_csv("test_label.csv")
print(y_train.shape)
print(y_test.shape)

(2672, 1)
(2348, 1)


In [12]:
print(x_train.shape)
print(x_test.shape)

(2672, 224, 224, 3)
(2348, 224, 224, 3)


In [13]:
y_train.iloc[1222]

Classes    Negative
Name: 1222, dtype: object

In [14]:
y_train=y_train.replace(to_replace="Negative",value=3)
y_train=y_train.replace(to_replace="Meningioma",value=1)
y_train=y_train.replace(to_replace="Glioma",value=0)
y_train=y_train.replace(to_replace="Pituitary",value=2)

In [15]:
y_train.iloc[1200]

Classes    3
Name: 1200, dtype: int64

In [16]:
y_train.head()

,Classes
0,0
1,0
2,0
3,0
4,0


In [17]:
y_train["Classes"].value_counts()

3    1032
0     791
2     489
1     360
Name: Classes, dtype: int64

In [18]:
from keras.utils import to_categorical
y_train =to_categorical(y_train, num_classes=4)
print(y_train.shape)

Using TensorFlow backend.


(2672, 4)


In [19]:
type(y_train)

numpy.ndarray

In [20]:
y_test=y_test.replace(to_replace="Negative",value=3)
y_test=y_test.replace(to_replace="Meningioma",value=1)
y_test=y_test.replace(to_replace="Glioma",value=0)
y_test=y_test.replace(to_replace="Pituitary",value=2)

In [21]:
y_test = to_categorical(y_test, num_classes=4)

In [22]:
y_test.shape

(2348, 4)

In [23]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, BatchNormalization
import numpy as np
import math, cv2, os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Activation, Dense, Conv2D, Reshape, concatenate, BatchNormalization, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import optimizers
from IPython.display import clear_output
%matplotlib inline
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from random import randint
from sklearn.utils import shuffle
import tensorflow as tf

In [24]:
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from tensorflow.keras.layers import Dropout

In [74]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
def build_model(hp):
    model = Sequential()
    model.add(Conv2D(hp.Int('filters', min_value=16, max_value=64, step=16), 
                     kernel_size=(3,3), padding='same', input_shape=(224, 224, 3), 
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(hp.Float('first_drop', min_value=0.2, max_value=0.3, step=0.05)))
    
    for i in range(hp.Int('num_layers', 1, 4)):
        model.add(Conv2D(hp.Int('conv_{i}', min_value=32, max_value=512, step=32), 
                         kernel_size=(3,3), padding='same', activation='relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(hp.Float('conv_drop', min_value=0.2, max_value=0.4, step=0.05)))
    
    model.add(Conv2D(512, 
                     kernel_size=(3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(hp.Float('dropout', min_value=0.25, max_value=0.5, step=0.1)))
    
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(units=256, activation='relu', 
                    kernel_initializer='he_uniform'))
    model.add(Dropout(hp.Float('final_drop', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(Dense(units=4, activation='sigmoid'))
    
    model.compile(optimizer=hp.Choice('opt', values=["Adam", "RMSprop", "AdaDelta"]),
                  loss='categorical_crossentropy', metrics=["accuracy"])
    
    return model

In [75]:
SEED = 10
dir='logs'

In [76]:
from kerastuner.tuners import Hyperband

In [77]:
turner = RandomSearch(build_model,
                      objective="val_accuracy",
                      max_trials=3,
                      seed=SEED,
                      directory=dir,
                      executions_per_trial=3)

INFO:tensorflow:Reloading Oracle from existing project logs\untitled_project\oracle.json


In [78]:
turner.search_space_summary()

In [82]:
turner.search(x_train, y_train,
             epochs=50,
             batch_size=24,
             validation_data=(x_test, y_test))

Train on 2672 samples, validate on 2348 samples
Epoch 1/50
2672/2672 [==============================] - ETA: 3:14 - loss: 1.5373 - accuracy: 0.21 - ETA: 1:51 - loss: 1.5362 - accuracy: 0.20 - ETA: 1:24 - loss: 1.5195 - accuracy: 0.22 - ETA: 1:10 - loss: 1.4912 - accuracy: 0.26 - ETA: 1:01 - loss: 1.5120 - accuracy: 0.25 - ETA: 55s - loss: 1.4848 - accuracy: 0.2708 - ETA: 51s - loss: 1.4706 - accuracy: 0.258 - ETA: 47s - loss: 1.4648 - accuracy: 0.253 - ETA: 44s - loss: 1.4540 - accuracy: 0.253 - ETA: 42s - loss: 1.4513 - accuracy: 0.259 - ETA: 40s - loss: 1.4432 - accuracy: 0.269 - ETA: 39s - loss: 1.4279 - accuracy: 0.273 - ETA: 37s - loss: 1.4383 - accuracy: 0.269 - ETA: 36s - loss: 1.4314 - accuracy: 0.270 - ETA: 35s - loss: 1.4261 - accuracy: 0.279 - ETA: 34s - loss: 1.4231 - accuracy: 0.283 - ETA: 33s - loss: 1.4210 - accuracy: 0.288 - ETA: 32s - loss: 1.4224 - accuracy: 0.284 - ETA: 31s - loss: 1.4178 - accuracy: 0.287 - ETA: 30s - loss: 1.4193 - accuracy: 0.287 - ETA: 29s - loss

2672/2672 [==============================] - ETA: 32s - loss: 0.9226 - accuracy: 0.718 - ETA: 32s - loss: 0.9657 - accuracy: 0.656 - ETA: 32s - loss: 1.0036 - accuracy: 0.604 - ETA: 31s - loss: 1.0240 - accuracy: 0.585 - ETA: 31s - loss: 1.0227 - accuracy: 0.587 - ETA: 30s - loss: 1.0066 - accuracy: 0.599 - ETA: 30s - loss: 1.0114 - accuracy: 0.607 - ETA: 29s - loss: 1.0082 - accuracy: 0.628 - ETA: 29s - loss: 1.0305 - accuracy: 0.621 - ETA: 29s - loss: 1.0248 - accuracy: 0.612 - ETA: 28s - loss: 1.0134 - accuracy: 0.622 - ETA: 28s - loss: 1.0117 - accuracy: 0.630 - ETA: 27s - loss: 1.0058 - accuracy: 0.634 - ETA: 27s - loss: 1.0053 - accuracy: 0.638 - ETA: 27s - loss: 1.0109 - accuracy: 0.633 - ETA: 26s - loss: 1.0134 - accuracy: 0.623 - ETA: 26s - loss: 1.0185 - accuracy: 0.614 - ETA: 25s - loss: 1.0140 - accuracy: 0.616 - ETA: 25s - loss: 1.0119 - accuracy: 0.613 - ETA: 25s - loss: 1.0109 - accuracy: 0.612 - ETA: 24s - loss: 1.0085 - accuracy: 0.613 - ETA: 24s - loss: 1.0076 - accur

2672/2672 [==============================] - ETA: 32s - loss: 0.8624 - accuracy: 0.656 - ETA: 32s - loss: 0.8637 - accuracy: 0.703 - ETA: 31s - loss: 0.8526 - accuracy: 0.687 - ETA: 31s - loss: 0.8573 - accuracy: 0.679 - ETA: 31s - loss: 0.8889 - accuracy: 0.662 - ETA: 30s - loss: 0.8667 - accuracy: 0.697 - ETA: 30s - loss: 0.8540 - accuracy: 0.714 - ETA: 30s - loss: 0.8510 - accuracy: 0.718 - ETA: 29s - loss: 0.8493 - accuracy: 0.715 - ETA: 29s - loss: 0.8451 - accuracy: 0.712 - ETA: 28s - loss: 0.8516 - accuracy: 0.710 - ETA: 28s - loss: 0.8545 - accuracy: 0.700 - ETA: 27s - loss: 0.8605 - accuracy: 0.692 - ETA: 27s - loss: 0.8609 - accuracy: 0.696 - ETA: 27s - loss: 0.8670 - accuracy: 0.685 - ETA: 26s - loss: 0.8632 - accuracy: 0.687 - ETA: 26s - loss: 0.8630 - accuracy: 0.689 - ETA: 25s - loss: 0.8645 - accuracy: 0.692 - ETA: 25s - loss: 0.8554 - accuracy: 0.702 - ETA: 25s - loss: 0.8502 - accuracy: 0.706 - ETA: 24s - loss: 0.8507 - accuracy: 0.712 - ETA: 24s - loss: 0.8544 - accur

2672/2672 [==============================] - ETA: 32s - loss: 0.6956 - accuracy: 0.781 - ETA: 32s - loss: 0.7487 - accuracy: 0.687 - ETA: 32s - loss: 0.7668 - accuracy: 0.697 - ETA: 32s - loss: 0.7743 - accuracy: 0.718 - ETA: 31s - loss: 0.7670 - accuracy: 0.718 - ETA: 31s - loss: 0.7696 - accuracy: 0.713 - ETA: 30s - loss: 0.7702 - accuracy: 0.705 - ETA: 30s - loss: 0.7659 - accuracy: 0.707 - ETA: 29s - loss: 0.7851 - accuracy: 0.697 - ETA: 29s - loss: 0.7935 - accuracy: 0.687 - ETA: 28s - loss: 0.7955 - accuracy: 0.690 - ETA: 28s - loss: 0.7855 - accuracy: 0.697 - ETA: 28s - loss: 0.7816 - accuracy: 0.709 - ETA: 27s - loss: 0.7915 - accuracy: 0.705 - ETA: 27s - loss: 0.7910 - accuracy: 0.710 - ETA: 26s - loss: 0.7991 - accuracy: 0.710 - ETA: 26s - loss: 0.8051 - accuracy: 0.704 - ETA: 25s - loss: 0.8059 - accuracy: 0.699 - ETA: 25s - loss: 0.8014 - accuracy: 0.703 - ETA: 25s - loss: 0.7975 - accuracy: 0.712 - ETA: 24s - loss: 0.7991 - accuracy: 0.714 - ETA: 24s - loss: 0.7950 - accur

2672/2672 [==============================] - ETA: 32s - loss: 0.7325 - accuracy: 0.718 - ETA: 31s - loss: 0.7730 - accuracy: 0.703 - ETA: 31s - loss: 0.7520 - accuracy: 0.739 - ETA: 31s - loss: 0.7585 - accuracy: 0.750 - ETA: 31s - loss: 0.7536 - accuracy: 0.737 - ETA: 30s - loss: 0.7507 - accuracy: 0.724 - ETA: 30s - loss: 0.7435 - accuracy: 0.736 - ETA: 29s - loss: 0.7579 - accuracy: 0.730 - ETA: 29s - loss: 0.7388 - accuracy: 0.746 - ETA: 29s - loss: 0.7265 - accuracy: 0.753 - ETA: 28s - loss: 0.7227 - accuracy: 0.755 - ETA: 28s - loss: 0.7290 - accuracy: 0.752 - ETA: 27s - loss: 0.7274 - accuracy: 0.750 - ETA: 27s - loss: 0.7303 - accuracy: 0.752 - ETA: 27s - loss: 0.7296 - accuracy: 0.752 - ETA: 26s - loss: 0.7261 - accuracy: 0.750 - ETA: 26s - loss: 0.7219 - accuracy: 0.753 - ETA: 25s - loss: 0.7228 - accuracy: 0.751 - ETA: 25s - loss: 0.7180 - accuracy: 0.754 - ETA: 25s - loss: 0.7134 - accuracy: 0.757 - ETA: 24s - loss: 0.7141 - accuracy: 0.751 - ETA: 24s - loss: 0.7123 - accur

...

2672/2672 [==============================] - ETA: 15s - loss: 1.1921e-07 - accuracy: 0.281 - ETA: 15s - loss: 1.1921e-07 - accuracy: 0.359 - ETA: 15s - loss: 1.1921e-07 - accuracy: 0.354 - ETA: 14s - loss: 1.1921e-07 - accuracy: 0.367 - ETA: 14s - loss: 1.1921e-07 - accuracy: 0.343 - ETA: 14s - loss: 1.1921e-07 - accuracy: 0.338 - ETA: 14s - loss: 1.1921e-07 - accuracy: 0.330 - ETA: 14s - loss: 1.1921e-07 - accuracy: 0.332 - ETA: 14s - loss: 1.1921e-07 - accuracy: 0.340 - ETA: 13s - loss: 1.1921e-07 - accuracy: 0.325 - ETA: 13s - loss: 1.1921e-07 - accuracy: 0.315 - ETA: 13s - loss: 1.1921e-07 - accuracy: 0.315 - ETA: 13s - loss: 1.1921e-07 - accuracy: 0.310 - ETA: 13s - loss: 1.1921e-07 - accuracy: 0.308 - ETA: 12s - loss: 1.1921e-07 - accuracy: 0.306 - ETA: 12s - loss: 1.1921e-07 - accuracy: 0.300 - ETA: 12s - loss: 1.1921e-07 - accuracy: 0.294 - ETA: 12s - loss: 1.1921e-07 - accuracy: 0.291 - ETA: 12s - loss: 1.1921e-07 - accuracy: 0.292 - ETA: 11s - loss: 1.1921e-07 - accuracy: 0.2

2672/2672 [==============================] - ETA: 15s - loss: 1.1921e-07 - accuracy: 0.312 - ETA: 15s - loss: 1.1921e-07 - accuracy: 0.281 - ETA: 15s - loss: 1.1921e-07 - accuracy: 0.291 - ETA: 14s - loss: 1.1921e-07 - accuracy: 0.281 - ETA: 14s - loss: 1.1921e-07 - accuracy: 0.281 - ETA: 14s - loss: 1.1921e-07 - accuracy: 0.291 - ETA: 14s - loss: 1.1921e-07 - accuracy: 0.303 - ETA: 14s - loss: 1.1921e-07 - accuracy: 0.296 - ETA: 14s - loss: 1.1921e-07 - accuracy: 0.305 - ETA: 13s - loss: 1.1921e-07 - accuracy: 0.309 - ETA: 13s - loss: 1.1921e-07 - accuracy: 0.304 - ETA: 13s - loss: 1.1921e-07 - accuracy: 0.309 - ETA: 13s - loss: 1.1921e-07 - accuracy: 0.314 - ETA: 13s - loss: 1.1921e-07 - accuracy: 0.308 - ETA: 12s - loss: 1.1921e-07 - accuracy: 0.308 - ETA: 12s - loss: 1.1921e-07 - accuracy: 0.310 - ETA: 12s - loss: 1.1921e-07 - accuracy: 0.314 - ETA: 12s - loss: 1.1921e-07 - accuracy: 0.310 - ETA: 12s - loss: 1.1921e-07 - accuracy: 0.314 - ETA: 12s - loss: 1.1921e-07 - accuracy: 0.3

2672/2672 [==============================] - ETA: 15s - loss: 1.1921e-07 - accuracy: 0.406 - ETA: 15s - loss: 1.1921e-07 - accuracy: 0.359 - ETA: 15s - loss: 1.1921e-07 - accuracy: 0.343 - ETA: 14s - loss: 1.1921e-07 - accuracy: 0.296 - ETA: 14s - loss: 1.1921e-07 - accuracy: 0.325 - ETA: 14s - loss: 1.1921e-07 - accuracy: 0.291 - ETA: 14s - loss: 1.1921e-07 - accuracy: 0.285 - ETA: 14s - loss: 1.1921e-07 - accuracy: 0.289 - ETA: 14s - loss: 1.1921e-07 - accuracy: 0.281 - ETA: 13s - loss: 1.1921e-07 - accuracy: 0.284 - ETA: 13s - loss: 1.1921e-07 - accuracy: 0.278 - ETA: 13s - loss: 1.1921e-07 - accuracy: 0.283 - ETA: 13s - loss: 1.1921e-07 - accuracy: 0.278 - ETA: 13s - loss: 1.1921e-07 - accuracy: 0.281 - ETA: 12s - loss: 1.1921e-07 - accuracy: 0.281 - ETA: 12s - loss: 1.1921e-07 - accuracy: 0.285 - ETA: 12s - loss: 1.1921e-07 - accuracy: 0.294 - ETA: 12s - loss: 1.1921e-07 - accuracy: 0.296 - ETA: 12s - loss: 1.1921e-07 - accuracy: 0.302 - ETA: 11s - loss: 1.1921e-07 - accuracy: 0.3

Train on 2672 samples, validate on 2348 samples
Epoch 1/50
  32/2672 [..............................] - ETA: 24:52WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


ResourceExhaustedError:  OOM when allocating tensor with shape[32,288,112,112] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential/conv2d_1/Conv2D (defined at C:\Users\KARTHIKEYAN\Anaconda3\envs\tf_gpu\lib\site-packages\kerastuner\engine\multi_execution_tuner.py:96) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_distributed_function_188946]

Function call stack:
distributed_function


In [84]:
turner.results_summary()

In [85]:
model = turner.get_best_models()[0]

In [87]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 96)      27744     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 96)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 56, 56, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 96)        8

In [88]:
model.save("cnn_model.h5")

In [90]:
turner.get_best_hyperparameters()[0].values

{'filters': 32,
 'first_drop': 0.25,
 'num_layers': 4,
 'conv_{i}': 96,
 'conv_drop': 0.39999999999999997,
 'dropout': 0.44999999999999996,
 'final_drop': 0.4000000000000001,
 'opt': 'RMSprop'}

In [91]:
hps = turner.oracle.get_best_trials(num_trials=1)[0].hyperparameters

In [92]:
model = build_model(hps)

In [93]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 112, 112, 96)      27744     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 56, 56, 96)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 56, 56, 96)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 56, 56, 96)       